<a href="https://colab.research.google.com/github/susantaghosh1/nlp-notebooks/blob/develop/Fine_Tuning_Deberta_Token_Classification%5BNER%5D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%capture
!pip install datasets transformers[sentencepiece]
!pip3 install torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu113
!pip install scipy sklearn
!pip install seqeval[gpu]

In [ ]:
%%capture
! pip install cloud-tpu-client==0.10 torch==1.9.0 https://storage.googleapis.com/tpu-pytorch/wheels/torch_xla-1.9-cp37-cp37m-linux_x86_64.whl
! pip install git+https://github.com/huggingface/accelerate

In [ ]:
from transformers import AutoTokenizer,AutoModelForTokenClassification

In [ ]:
import torch
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
device

device(type='cpu')

In [ ]:
!nvidia-smi

Wed Jul 13 04:36:35 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   39C    P8     9W /  70W |      3MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## Load the dataset

In [ ]:
from datasets import load_dataset
raw_datasets = load_dataset("conll2003")

Generating train split:   0%|          | 0/14042 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3251 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/3454 [00:00<?, ? examples/s]

Dataset conll2003 downloaded and prepared to /root/.cache/huggingface/datasets/conll2003/conll2003/1.0.0/63f4ebd1bcb7148b1644497336fd74643d4ce70123334431a3c053b7ee4e96ee. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 14042
    })
    validation: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 3251
    })
    test: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 3454
    })
})

In [ ]:
raw_datasets['train'][0]

{'chunk_tags': [11, 21, 11, 12, 21, 22, 11, 12, 0],
 'id': '0',
 'ner_tags': [3, 0, 7, 0, 0, 0, 7, 0, 0],
 'pos_tags': [22, 42, 16, 21, 35, 37, 16, 21, 7],
 'tokens': ['EU',
  'rejects',
  'German',
  'call',
  'to',
  'boycott',
  'British',
  'lamb',
  '.']}

In [ ]:
raw_datasets['train'].features['ner_tags'],raw_datasets['train'].features['ner_tags'].feature.names

(Sequence(feature=ClassLabel(num_classes=9, names=['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-MISC', 'I-MISC'], id=None), length=-1, id=None),
 ['O',
  'B-PER',
  'I-PER',
  'B-ORG',
  'I-ORG',
  'B-LOC',
  'I-LOC',
  'B-MISC',
  'I-MISC'])

In [ ]:
label_names = raw_datasets['train'].features['ner_tags'].feature.names
label_names

['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-MISC', 'I-MISC']

In [ ]:
sample_tokens = raw_datasets['train'][0]['tokens']
sample_tags = raw_datasets['train'][0]['ner_tags']
sample_tokens,sample_tags
s=""
y=""
for tokens,tags in zip(sample_tokens,sample_tags):
  s+=tokens+" "
  y+=label_names[tags]+" "
s,y

('EU rejects German call to boycott British lamb . ',
 'B-ORG O B-MISC O O O B-MISC O O ')

In [ ]:
checkpoint_name = "microsoft/deberta-v3-base"
#checkpoint_name = "bert-base-cased"
tokeinzer = AutoTokenizer.from_pretrained(checkpoint_name)
tokeinzer.is_fast

Downloading:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/579 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.35M [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
/usr/local/lib/python3.7/dist-packages/transformers/convert_slow_tokenizer.py:435: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  "The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option"
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


True

In [ ]:
tokenized_text = tokeinzer(raw_datasets['train'][0]['tokens'],is_split_into_words=True)
print(tokenized_text)
print(tokenized_text.tokens())
print(tokeinzer.decode(tokenized_text['input_ids']))
print(tokenized_text.word_ids())

{'input_ids': [1, 2805, 27144, 2324, 660, 264, 20007, 1668, 12649, 323, 2], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}
['[CLS]', '▁EU', '▁rejects', '▁German', '▁call', '▁to', '▁boycott', '▁British', '▁lamb', '▁.', '[SEP]']
[CLS] EU rejects German call to boycott British lamb.[SEP]
[None, 0, 1, 2, 3, 4, 5, 6, 7, 8, None]


In [ ]:
sample_tags,tokenized_text.word_ids()

([3, 0, 7, 0, 0, 0, 7, 0, 0], [None, 0, 1, 2, 3, 4, 5, 6, 7, 8, None])

In [ ]:
new_labels =[]
for word_id in tokenized_text.word_ids():
  if word_id is None:
    new_labels.append(-100)
  else:
    new_labels.append(sample_tags[word_id])
new_labels

[-100, 3, 0, 7, 0, 0, 0, 7, 0, 0, -100]

In [ ]:
[-100, 3, 0, 7, 0, 0, 0, 7, 0, 0, 0, -100]

[-100, 3, 0, 7, 0, 0, 0, 7, 0, 0, 0, -100]

In [ ]:
def align_labels_with_tokens(labels,word_ids):
  new_labels = []
  current_word = None
  for word_id in word_ids:
    if word_id!=current_word:
      #start of a new token
      current_word=word_id
      label = -100 if word_id is None else labels[word_id]
      new_labels.append(label)
    elif word_id is None:
      new_labels.append(-100)
    else:
      # same word as previous token
      label = labels[word_id]
      if label %2 ==1: # all B- tokens are in odd position, I- tokens are in even position. So If the label is B-XXX we change it to I-XXX
        label+=1
      new_labels.append(label)
  return new_labels

In [ ]:
align_labels_with_tokens(sample_tags,tokenized_text.word_ids())

[-100, 3, 0, 7, 0, 0, 0, 7, 0, 0, -100]

In [ ]:
def tokenize_and_align_labels(batch_of_samples):
  batch_encoding = tokeinzer(batch_of_samples['tokens'],is_split_into_words=True,truncation=True,padding="max_length",max_length=256)
  batch_ner_tags = batch_of_samples['ner_tags']
  new_labels = []
  for i,labels in enumerate(batch_ner_tags):
    word_ids = batch_encoding.word_ids(i)
    new_labels.append(align_labels_with_tokens(labels,word_ids))
  batch_encoding['labels'] = new_labels
  return batch_encoding

In [ ]:
tokenizd_dataset = raw_datasets.map(tokenize_and_align_labels,batched=True,remove_columns = raw_datasets["train"].column_names)

Parameter 'function'=<function tokenize_and_align_labels at 0x7f65033f7b00> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


  0%|          | 0/15 [00:00<?, ?ba/s]

  0%|          | 0/4 [00:00<?, ?ba/s]

  0%|          | 0/4 [00:00<?, ?ba/s]

In [ ]:
tokenizd_dataset,len(tokenizd_dataset['train']['input_ids'])

(DatasetDict({
    train: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 14042
    })
    validation: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 3251
    })
    test: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 3454
    })
}),
 14042)

##Fine Tuning


In [ ]:
## metrics
from datasets import load_metric
metric = load_metric("seqeval")

This metric does not behave like the standard accuracy: it will actually take the lists of labels as strings, not integers, so we will need to fully decode the predictions and labels before passing them to the metric. Let’s see how it works. First, we’ll get the labels for our first training example:

In [ ]:
labels = raw_datasets["train"][0:3]["ner_tags"]
labels

[[3, 0, 7, 0, 0, 0, 7, 0, 0], [1, 2], [5, 0]]

In [ ]:
label_names
labels = [[label_names[l] for l in label] for label in labels]
labels

[['B-ORG', 'O', 'B-MISC', 'O', 'O', 'O', 'B-MISC', 'O', 'O'],
 ['B-PER', 'I-PER'],
 ['B-LOC', 'O']]

In [ ]:
preds = labels.copy()
preds[0][2] = "O"
preds

[['B-ORG', 'O', 'O', 'O', 'O', 'O', 'B-MISC', 'O', 'O'],
 ['B-PER', 'I-PER'],
 ['B-LOC', 'O']]

In [ ]:
metric.compute??

In [ ]:
metric.compute(predictions=[preds], references=[labels])

/usr/local/lib/python3.7/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: ['B-ORG', 'O', 'O', 'O', 'O', 'O', 'B-MISC', 'O', 'O'] seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.7/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: ['B-PER', 'I-PER'] seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.7/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: ['B-LOC', 'O'] seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))


{"LOC', 'O']": {'f1': 1.0, 'number': 1, 'precision': 1.0, 'recall': 1.0},
 "ORG', 'O', 'O', 'O', 'O', 'O', 'B-MISC', 'O', 'O']": {'f1': 1.0,
  'number': 1,
  'precision': 1.0,
  'recall': 1.0},
 "PER', 'I-PER']": {'f1': 1.0, 'number': 1, 'precision': 1.0, 'recall': 1.0},
 'overall_accuracy': 1.0,
 'overall_f1': 1.0,
 'overall_precision': 1.0,
 'overall_recall': 1.0}

In [ ]:
label_names

['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-MISC', 'I-MISC']

In [ ]:
id2label = {str(i):label for i,label in enumerate(label_names)}
label2id = {label:str(i) for i,label in enumerate(label_names)}
id2label,label2id

({'0': 'O',
  '1': 'B-PER',
  '2': 'I-PER',
  '3': 'B-ORG',
  '4': 'I-ORG',
  '5': 'B-LOC',
  '6': 'I-LOC',
  '7': 'B-MISC',
  '8': 'I-MISC'},
 {'B-LOC': '5',
  'B-MISC': '7',
  'B-ORG': '3',
  'B-PER': '1',
  'I-LOC': '6',
  'I-MISC': '8',
  'I-ORG': '4',
  'I-PER': '2',
  'O': '0'})

In [ ]:
from torch.utils.data import Dataset
import torch
class NERDataset(Dataset):
  def __init__(self,tokenized_dataset):
    self.data = tokenized_dataset
  def __getitem__(self, idx):
        print(idx)
        item = {key: torch.tensor(val) for key, val in self.data[idx].items()}
        return item
  def __len__(self):
        return len(self.data)


In [ ]:
tokenizd_dataset['train'][0].keys()

dict_keys(['input_ids', 'token_type_ids', 'attention_mask', 'labels'])

In [ ]:
train_dataset = NERDataset(tokenizd_dataset['train'])
validation_dataset = NERDataset(tokenizd_dataset['validation'])
test_dataset = NERDataset(tokenizd_dataset['test'])

## all the datasets will be accessed through index like train_dataset[1]


In [ ]:
from torch.utils.data import DataLoader
train_dataloader = DataLoader(train_dataset, batch_size=8, shuffle=True)
validation_dataloader = DataLoader(validation_dataset, batch_size=8, shuffle=False)
test_dataloader = DataLoader(test_dataset, batch_size=8, shuffle=False)

In [ ]:
model = AutoModelForTokenClassification.from_pretrained(checkpoint_name, id2label=id2label,label2id=label2id,)

Downloading:   0%|          | 0.00/354M [00:00<?, ?B/s]

Some weights of the model checkpoint at microsoft/deberta-v3-base were not used when initializing DebertaV2ForTokenClassification: ['lm_predictions.lm_head.LayerNorm.weight', 'lm_predictions.lm_head.dense.bias', 'mask_predictions.dense.bias', 'lm_predictions.lm_head.dense.weight', 'mask_predictions.LayerNorm.weight', 'mask_predictions.LayerNorm.bias', 'deberta.embeddings.position_embeddings.weight', 'lm_predictions.lm_head.LayerNorm.bias', 'lm_predictions.lm_head.bias', 'mask_predictions.classifier.bias', 'mask_predictions.classifier.weight', 'mask_predictions.dense.weight']
- This IS expected if you are initializing DebertaV2ForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DebertaV2ForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a Be

In [ ]:
for batch in train_dataloader:
  print({k:v.shape for k,v in batch.items()})
  outputs=model(**batch) ##one sample forward pass
  break

11085
7510
8012
9859
1405
3793
5231
2597
{'input_ids': torch.Size([8, 256]), 'token_type_ids': torch.Size([8, 256]), 'attention_mask': torch.Size([8, 256]), 'labels': torch.Size([8, 256])}


In [ ]:
outputs.logits.shape

torch.Size([8, 256, 9])

In [ ]:
preds = outputs.logits.detach().argmax(dim=-1)
preds.shape,batch['labels'].shape

(torch.Size([8, 256]), torch.Size([8, 256]))

In [ ]:
from torch.utils.data import DataLoader
train_dataloader = DataLoader(train_dataset, batch_size=8, shuffle=True)
validation_dataloader = DataLoader(validation_dataset, batch_size=8, shuffle=False)
test_dataloader = DataLoader(test_dataset, batch_size=8, shuffle=False)
model = AutoModelForTokenClassification.from_pretrained(checkpoint_name, id2label=id2label,label2id=label2id,)
from torch.optim import AdamW
optimizer = AdamW(model.parameters(),lr=2e-5)

Some weights of the model checkpoint at microsoft/deberta-v3-base were not used when initializing DebertaV2ForTokenClassification: ['lm_predictions.lm_head.LayerNorm.weight', 'lm_predictions.lm_head.dense.bias', 'mask_predictions.dense.bias', 'lm_predictions.lm_head.dense.weight', 'mask_predictions.LayerNorm.weight', 'mask_predictions.LayerNorm.bias', 'deberta.embeddings.position_embeddings.weight', 'lm_predictions.lm_head.LayerNorm.bias', 'lm_predictions.lm_head.bias', 'mask_predictions.classifier.bias', 'mask_predictions.classifier.weight', 'mask_predictions.dense.weight']
- This IS expected if you are initializing DebertaV2ForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DebertaV2ForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a Be

check whether the data is created correctly by fetching the first batch from the data loader, and then printing out the shapes of the tensors, decoding the input_ids back to text, etc.

In [ ]:

def training_function():
  from accelerate import Accelerator
  accelerator  = Accelerator()
  model, optimizer, train_dataloader, eval_dataloader = accelerator.prepare( model, optimizer, train_dataloader, validation_dataloader)
  from transformers import get_scheduler
  num_train_epochs = 3
  num_update_steps_per_epoch = len(train_dataloader)
  num_training_steps = num_train_epochs * num_update_steps_per_epoch
  lr_scheduler = get_scheduler("linear",optimizer=optimizer, num_warmup_steps=0,
    num_training_steps=num_training_steps,)

BertForTokenClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(28996, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwis